<a href="https://colab.research.google.com/github/swiminthewind/colab/blob/main/231125-%E8%BF%87%E6%8B%9F%E5%90%88%E6%A3%80%E6%9F%A5_HANTS%E4%B8%83%E9%98%B6RMSE%E3%80%81MAE%E3%80%81R2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [4]:
import pandas as pd
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

In [5]:
roi = ee.FeatureCollection("projects/ee-grn/assets/cd_plain_cd")
csPlus= ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
s2sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

In [6]:
timeField = 'system:time_start'
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.6

def cloud_remove(img):
    return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).select('.*').copyProperties(img, ['system:time_start'])

composite = s2sr.filterBounds(roi).filterDate('2022-01-01','2023-05-01').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)

def s2addVariables(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI')).addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

filteredSentinel = composite.map(s2addVariables)
print(filteredSentinel.size().getInfo())

873


In [7]:
dataset = filteredSentinel.select(['NDVI','t','constant'])
independents = ee.List(['constant', 't'])
dependent = ee.String('NDVI')

In [8]:
# 七阶谐波
harmonicIndependents_7 = ee.List(['constant', 't', 'cos1', 'sin1', 'cos2', 'sin2', 'cos3', 'sin3', 'cos4', 'sin4', 'cos5', 'sin5', 'cos6', 'sin6', 'cos7', 'sin7'])

def add_sincos_7(image):
  timeRadians1 = image.select('t').multiply(2 * math.pi)
  timeRadians2 = image.select('t').multiply(4 * math.pi)
  timeRadians3 = image.select('t').multiply(6 * math.pi)
  timeRadians4 = image.select('t').multiply(8 * math.pi)
  timeRadians5 = image.select('t').multiply(10 * math.pi)
  timeRadians6 = image.select('t').multiply(12 * math.pi)
  timeRadians7 = image.select('t').multiply(14 * math.pi)
  return image.addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1')).addBands(timeRadians2.cos().rename('cos2')).addBands(timeRadians2.sin().rename('sin2')).addBands(timeRadians3.cos().rename('cos3')).addBands(timeRadians3.sin().rename('sin3')).addBands(timeRadians4.cos().rename('cos4')).addBands(timeRadians4.sin().rename('sin4')).addBands(timeRadians5.cos().rename('cos5')).addBands(timeRadians5.sin().rename('sin5')).addBands(timeRadians6.cos().rename('cos6')).addBands(timeRadians6.sin().rename('sin6')).addBands(timeRadians7.cos().rename('cos7')).addBands(timeRadians7.sin().rename('sin7'))

harmonicSentinel_7 = dataset.map(add_sincos_7)

harmonicTrend_7 = harmonicSentinel_7.select(harmonicIndependents_7.add(dependent)).reduce(ee.Reducer.linearRegression(numX=harmonicIndependents_7.length(),numY=1))

harmonicTrendCoefficients_7 = harmonicTrend_7.select('coefficients').arrayProject([0]).arrayFlatten([harmonicIndependents_7])

def fit_7(image):
  return image.addBands(image.select(harmonicIndependents_7).multiply(harmonicTrendCoefficients_7).reduce('sum').rename('fitted'))

fittedHarmonic_7 = harmonicSentinel_7.map(fit_7)

In [9]:
composite1 = s2sr.filterBounds(roi).filterDate('2022-01-01','2023-11-24').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)
filteredSentinel1 = composite1.map(s2addVariables)
dataset1 = filteredSentinel1.select(['NDVI','t','constant']);
harmonicSentinel1 = dataset1.map(add_sincos_7)
fittedHarmonic1 = harmonicSentinel1.map(fit_7)

In [10]:
points = [ee.Geometry.Point(103.69442853431933, 30.551562081391808),ee.Geometry.Point(103.65488522617737, 30.533297603476893),ee.Geometry.Point(103.59582889182555, 30.52539161730913),ee.Geometry.Point(103.56952465471126, 30.499742868049715),ee.Geometry.Point(103.53194781013387, 30.53031446487214),ee.Geometry.Point(103.50069397085026, 30.55269915216044),ee.Geometry.Point(103.46811131945273, 30.532325519447248),ee.Geometry.Point(103.49599853744098, 30.5045720744906),ee.Geometry.Point(103.53704010353692, 30.476564460773904),ee.Geometry.Point(103.56092304877916, 30.447647190442307),ee.Geometry.Point(103.59226607008384, 30.446978325284288),ee.Geometry.Point(103.62079985772485, 30.439781336184026),ee.Geometry.Point(103.59236417030701, 30.470914779739182),ee.Geometry.Point(103.68473444862916, 30.49993015029396),ee.Geometry.Point(103.75471558056192, 30.43588854096437),ee.Geometry.Point(103.84447728476779, 30.443179171186756),ee.Geometry.Point(103.85018047501515, 30.467788949900722),ee.Geometry.Point(103.6853230499682, 30.687827750484054),ee.Geometry.Point(103.65355195496241, 30.710867925627216),ee.Geometry.Point(103.6298830465707, 30.742799548270945),ee.Geometry.Point(103.59532947250756, 30.768805025614643),ee.Geometry.Point(103.56548916825788, 30.793976651061346),ee.Geometry.Point(103.72427329767005, 30.713917950747774),ee.Geometry.Point(103.74967233727047, 30.683810100434908),ee.Geometry.Point(103.71968488268605, 30.600692456798672),ee.Geometry.Point(103.74770587370591, 30.576323469541602),ee.Geometry.Point(103.74953856423888, 30.553011289234185),ee.Geometry.Point(103.7740948337402, 30.5171868713708),ee.Geometry.Point(103.81560460544674, 30.52750077210742),ee.Geometry.Point(103.81192584707763, 30.56217028279795),ee.Geometry.Point(103.81033394800156, 30.575315712703514),ee.Geometry.Point(103.77831760243781, 30.573233312511555),ee.Geometry.Point(103.91586749263347, 30.848234992679675),ee.Geometry.Point(103.85799727916184, 30.84885926682715),ee.Geometry.Point(103.85485361291917, 30.80372870506523),ee.Geometry.Point(103.79825870234883, 30.829515686457306),ee.Geometry.Point(103.74891874919244, 30.8550618763925),ee.Geometry.Point(103.79212743840034, 30.902011751384222),ee.Geometry.Point(103.82710462706356, 30.885321336141182),ee.Geometry.Point(103.85819793870925, 30.90270291204751),ee.Geometry.Point(103.82015288852125, 30.935334613556623),ee.Geometry.Point(103.75618708390955, 30.985240872546797),ee.Geometry.Point(103.75686040816862, 30.939191735967864),ee.Geometry.Point(103.7038372375421, 30.93940131371737),ee.Geometry.Point(103.88234843001472, 31.089717610229034),ee.Geometry.Point(103.95180338802317, 31.09406077465508),ee.Geometry.Point(103.95662367626194, 31.122732794428746),ee.Geometry.Point(104.00554001481827, 31.09055592122708),ee.Geometry.Point(104.03197802496413, 31.058004483536926),ee.Geometry.Point(103.98008746600517, 31.060274165973127),ee.Geometry.Point(103.9804218985842, 31.010140492829244),ee.Geometry.Point(104.03809591160946, 31.00245746171415),ee.Geometry.Point(104.07186914298828, 31.02673280784911),ee.Geometry.Point(104.10680619974202, 31.003808569333344),ee.Geometry.Point(104.10340390563823, 30.972006260620166),ee.Geometry.Point(104.1242011529515, 30.924481161592546),ee.Geometry.Point(104.07048682166169, 30.936092660735714),ee.Geometry.Point(104.04891369076513, 30.929791950947198),ee.Geometry.Point(104.03893868170857, 30.95408959258743),ee.Geometry.Point(104.03782836554626, 30.901454363752542),ee.Geometry.Point(104.07251125353997, 30.899523572996404),ee.Geometry.Point(104.12622558482975, 30.876773239421727),ee.Geometry.Point(104.16596955251913, 30.86185754639795),ee.Geometry.Point(104.19711637337745, 30.843508345563016),ee.Geometry.Point(104.20473697707779, 30.787947946437082),ee.Geometry.Point(104.2233002147633, 30.74701339876645),ee.Geometry.Point(104.32238144017084, 30.867235222268405),ee.Geometry.Point(104.34800343482397, 30.86287868053919),ee.Geometry.Point(104.32160109748651, 30.792647838947417),ee.Geometry.Point(103.68982228293112, 30.47573060887691),ee.Geometry.Point(103.682803657873, 30.42125377130696),ee.Geometry.Point(103.65444377517308, 30.390432464825544),ee.Geometry.Point(103.62306954016104, 30.394566051502085),ee.Geometry.Point(103.43668803432911, 30.473443090036486),ee.Geometry.Point(103.56000447396217, 30.390294232692884),ee.Geometry.Point(103.67927204983867, 30.311903236173308),ee.Geometry.Point(103.76919428168246, 30.41811010506428),ee.Geometry.Point(103.68609447445044, 30.57786631850608),ee.Geometry.Point(103.58707121735658, 30.56275888413701),ee.Geometry.Point(103.62182991006817, 30.5831235986481),ee.Geometry.Point(103.61745107283369, 30.616794270702663),ee.Geometry.Point(103.56919022213226, 30.62262677488058),ee.Geometry.Point(103.59710419472684, 30.66326256278062),ee.Geometry.Point(103.63035125218131, 30.69151988615631),ee.Geometry.Point(103.62576729629838, 30.63486254817123),ee.Geometry.Point(103.6501897927681, 30.606542797380797),ee.Geometry.Point(103.6639639559222, 30.67538685854495),ee.Geometry.Point(103.81523004095824, 30.638768720694053),ee.Geometry.Point(103.87737653234012, 30.7408687575148),ee.Geometry.Point(103.90792137455621, 30.738166542276414),ee.Geometry.Point(103.81904703145999, 30.766843021151118),ee.Geometry.Point(103.63658061635299, 30.784545652333307),ee.Geometry.Point(103.62725217694917, 30.828244842657078),ee.Geometry.Point(103.62660560729641, 30.88052334340767),ee.Geometry.Point(103.632411356868, 30.939231867877343),ee.Geometry.Point(103.67458107553044, 31.064180338495948),ee.Geometry.Point(103.70539346380976, 31.035976524332895),ee.Geometry.Point(104.01145724191619, 30.97983644207001),ee.Geometry.Point(103.98222783451084, 30.95264930294715),ee.Geometry.Point(103.9124830349339, 30.953741782705258),ee.Geometry.Point(103.85152712353329, 30.95874043498617),ee.Geometry.Point(103.79579727856733, 30.96167452347934),ee.Geometry.Point(103.78737403667736, 31.040792353470632),ee.Geometry.Point(103.85454147584541, 31.06187052415026),ee.Geometry.Point(103.88376196504863, 31.045844514964173),ee.Geometry.Point(103.84999319277091, 31.0103456114777),ee.Geometry.Point(103.91530118679968, 31.007634478037204),ee.Geometry.Point(103.97618575258342, 30.898966185364717),ee.Geometry.Point(103.94299666344261, 30.873669705088535),ee.Geometry.Point(103.97305992274497, 30.848172565264928),ee.Geometry.Point(103.99353165568132, 30.86875577572763),ee.Geometry.Point(104.04493617262546, 30.849260585921964),ee.Geometry.Point(104.10052778545875, 30.837296817795565),ee.Geometry.Point(104.12120909614465, 30.817378013389817),ee.Geometry.Point(103.84901664964019, 31.010283184062946),ee.Geometry.Point(103.71719224519717, 30.472814356787953),ee.Geometry.Point(103.68974647821322, 30.475694936068486),ee.Geometry.Point(103.6548450942679, 30.472595860836336),ee.Geometry.Point(103.65476483044894, 30.472903538809025),ee.Geometry.Point(103.651349159042, 30.446982784385344),ee.Geometry.Point(103.5293481542197, 30.420950552435325),ee.Geometry.Point(103.4988523621152, 30.450113073324868),ee.Geometry.Point(103.4963240518179, 30.47491905248518),ee.Geometry.Point(103.49697062147064, 30.474923511586237),ee.Geometry.Point(103.44297090771342, 30.503831863715725),ee.Geometry.Point(103.436179696809, 30.47350105835019),ee.Geometry.Point(103.43450307481291, 30.42161049939123),ee.Geometry.Point(103.46724625384837, 30.446911438768485),ee.Geometry.Point(103.61455711625003, 30.36449387399765),ee.Geometry.Point(103.68620595197677, 30.358759470042916),ee.Geometry.Point(103.71908290404384, 30.3600927412579),ee.Geometry.Point(103.77645815729852, 30.389710090454876),ee.Geometry.Point(103.77456303935081, 30.390454760330815),ee.Geometry.Point(103.8763241844915, 30.416678733626124),ee.Geometry.Point(103.87105798614739, 30.445970568446214),ee.Geometry.Point(103.68606326074307, 30.577705790868162),ee.Geometry.Point(103.65106823567564, 30.55970885901645),ee.Geometry.Point(103.62362246869168, 30.550754984101136),ee.Geometry.Point(103.59622129271824, 30.715059480617448),ee.Geometry.Point(103.59917321761561, 30.713730668503533),ee.Geometry.Point(103.63093985352037, 30.691537722560533),ee.Geometry.Point(103.90764936939195, 30.73843408833962),ee.Geometry.Point(103.84272485805378, 30.733974987286174),ee.Geometry.Point(103.81973373302223, 30.766521965875278),ee.Geometry.Point(103.70528198628341, 31.036101379162403),ee.Geometry.Point(103.70293204002824, 30.93987397842904),ee.Geometry.Point(103.7489321264956, 30.855146599312512),ee.Geometry.Point(103.82210597478266, 30.875426590903576),ee.Geometry.Point(103.85075569905104, 30.906560034458742),ee.Geometry.Point(103.91618408880825, 30.901927028464208),ee.Geometry.Point(103.98263807180777, 30.952635925644),ee.Geometry.Point(103.85198195184073, 30.958967849139892),ee.Geometry.Point(103.75965626452916, 31.013016613008702),ee.Geometry.Point(103.78707081780573, 31.040948422007496),ee.Geometry.Point(103.85476888999914, 31.061754587522877),ee.Geometry.Point(103.88393586998974, 31.045719660134687),ee.Geometry.Point(103.84999765187194, 31.010109279121863),ee.Geometry.Point(103.91554197825657, 31.007839596685663),ee.Geometry.Point(103.94347824635639, 30.87328176329688),ee.Geometry.Point(103.99313925478862, 30.868800366738167),ee.Geometry.Point(104.04520817778972, 30.849577182096763),ee.Geometry.Point(104.12120909614465, 30.81684292126341),ee.Geometry.Point(103.86281310829955, 30.532307683043033),ee.Geometry.Point(103.88304404977903, 30.514056582431277),ee.Geometry.Point(104.32233239005927, 30.893909564770112),ee.Geometry.Point(103.7171610314898, 30.54972493175779),ee.Geometry.Point(103.4787774891726, 30.511684340670843),ee.Geometry.Point(103.74441505712846, 30.49009337337006),ee.Geometry.Point(103.73493054918778, 30.66060493855278),ee.Geometry.Point(103.53428437908589, 30.77065555255182),ee.Geometry.Point(103.69229262491474, 30.769362413246316),ee.Geometry.Point(103.69146769121986, 30.7438385188164),ee.Geometry.Point(103.78781548768166, 30.60887490723174),ee.Geometry.Point(104.0033996463126, 30.766624525199504),ee.Geometry.Point(103.91834675281916, 30.896687584726415),ee.Geometry.Point(103.94983246535755, 31.043904806005933),ee.Geometry.Point(103.71711644047926, 30.472925834314296),ee.Geometry.Point(103.65482279876264, 30.472685042857403),ee.Geometry.Point(103.49845550212143, 30.450358323882796),ee.Geometry.Point(103.49697062147064, 30.474923511586237),ee.Geometry.Point(103.43403486920232, 30.422065327698693),ee.Geometry.Point(103.46697870778517, 30.44713439382116),ee.Geometry.Point(103.68610339265254, 30.35813965499649),ee.Geometry.Point(103.71580992387062, 30.359718176769412),ee.Geometry.Point(103.77456303935081, 30.390454760330815),ee.Geometry.Point(103.83757459633706, 30.3949673705969),ee.Geometry.Point(103.87085286749893, 30.44555587204824),ee.Geometry.Point(103.71971163729236, 30.600317892310187),ee.Geometry.Point(103.653436018335, 30.561893818532642),ee.Geometry.Point(103.84408934297613, 30.73422915604622),ee.Geometry.Point(103.59517340397068, 30.85430382921341),ee.Geometry.Point(103.60227229284777, 30.991452400314245),ee.Geometry.Point(103.76001745171446, 31.012842708067623),ee.Geometry.Point(103.84697884045877, 30.60314050327702),ee.Geometry.Point(103.85057733500891, 30.543642717920893),ee.Geometry.Point(103.87578017416297, 30.416518205988194),ee.Geometry.Point(103.68563518704195, 30.82783460536016),ee.Geometry.Point(103.88021697971115, 30.87170770062501),ee.Geometry.Point(103.99598862036179, 30.746968807755923),ee.Geometry.Point(103.56322840402379, 30.775614072923254),ee.Geometry.Point(103.71968488268605, 30.53063552014799),ee.Geometry.Point(103.64892786716997, 30.445399803511375),ee.Geometry.Point(103.61276009852548, 30.363865140749123),ee.Geometry.Point(103.52937936792709, 30.42100406164797),ee.Geometry.Point(103.62266822106623, 30.551031448366444),ee.Geometry.Point(103.9388318630587, 30.818474952248977),ee.Geometry.Point(104.26909072348113, 30.819763632453423),ee.Geometry.Point(103.62424228373808, 30.52061146097984),ee.Geometry.Point(103.44218164682695, 30.55269915216044),ee.Geometry.Point(103.93617423883086, 30.68545996782468),ee.Geometry.Point(103.59622129271824, 30.715059480617448),ee.Geometry.Point(104.08844362160391, 30.78663251162632),ee.Geometry.Point(103.8830485088801, 30.513815790974384),ee.Geometry.Point(103.99141358268095, 30.704210487754427),ee.Geometry.Point(104.10265923576232, 30.575364762815095),ee.Geometry.Point(103.6651233221961, 30.768189669669265),ee.Geometry.Point(103.08202451384118, 30.272961906673565),ee.Geometry.Point(103.2051893440384, 30.270380087163616),ee.Geometry.Point(103.38279533899716, 30.321771226804582),ee.Geometry.Point(103.71164066438563, 30.294740156218595),ee.Geometry.Point(103.98782846543396, 30.283364989431256),ee.Geometry.Point(104.117650733504, 30.27742546682806),ee.Geometry.Point(103.20908213925807, 30.39105227987197),ee.Geometry.Point(103.33590343231911, 30.411555226515723),ee.Geometry.Point(103.51891385775464, 30.379418485223535),ee.Geometry.Point(103.69645742529865, 30.37075445187668),ee.Geometry.Point(103.80408228832464, 30.39931499412401),ee.Geometry.Point(103.98430577560175, 30.384430514807605),ee.Geometry.Point(104.15844704904197, 30.383016979773657),ee.Geometry.Point(104.12181999298896, 30.476341505721233),ee.Geometry.Point(103.94307246816052, 30.510948588997028),ee.Geometry.Point(103.81990317886225, 30.509704499803117),ee.Geometry.Point(103.63615254265184, 30.499912313889737),ee.Geometry.Point(103.52109881727085, 30.516910407105488),ee.Geometry.Point(103.36054888384152, 30.511068984725462),ee.Geometry.Point(103.22142939007506, 30.53893390720845),ee.Geometry.Point(103.2192622669631, 30.663467681429083),ee.Geometry.Point(103.36450856557698, 30.644231119484523),ee.Geometry.Point(103.49705980349171, 30.643936818814993),ee.Geometry.Point(103.68750355038334, 30.664952562079883),ee.Geometry.Point(103.80212920206321, 30.632329778772874),ee.Geometry.Point(103.99826722100008, 30.649622172658137),ee.Geometry.Point(104.14486908633424, 30.629391231178655),ee.Geometry.Point(104.38531273333813, 30.732169051359524),ee.Geometry.Point(104.2738530434062, 30.787399477007504),ee.Geometry.Point(104.13134463283912, 30.766744920927948),ee.Geometry.Point(103.83980860596482, 30.76384204614215),ee.Geometry.Point(103.67305160386911, 30.764252283439063),ee.Geometry.Point(103.49113811729273, 30.769509563581085),ee.Geometry.Point(103.34861187032143, 30.77504776708946),ee.Geometry.Point(103.21975722718001, 30.78279768472035),ee.Geometry.Point(103.09000184562579, 30.764805211969694),ee.Geometry.Point(103.51827620630401, 30.878521207034673),ee.Geometry.Point(103.65610256076496, 30.890814948639026),ee.Geometry.Point(103.85091176758792, 30.89502434003347),ee.Geometry.Point(104.25732761490214, 30.87454814799606),ee.Geometry.Point(104.10342174204246, 30.896821357758018),ee.Geometry.Point(104.08130014171631, 30.986097019949057),ee.Geometry.Point(103.99985020187405, 30.984897521765678),ee.Geometry.Point(103.81098497675535, 31.022416398029367),ee.Geometry.Point(103.69272069861587, 31.000245747591638),ee.Geometry.Point(103.54900387166332, 31.00242624800677),ee.Geometry.Point(103.57515649934176, 31.125305695736575),ee.Geometry.Point(103.72377387835206, 31.117720764844666),ee.Geometry.Point(103.82175370579944, 31.12052553940729),ee.Geometry.Point(103.95845190769386, 31.12639817549468),ee.Geometry.Point(103.85190168802178, 31.252412371265066),ee.Geometry.Point(103.65502791741109, 31.27065009457365),ee.Geometry.Point(103.87428637531008, 31.33891893170191),ee.Geometry.Point(103.36623869678571, 30.41068124270924),ee.Geometry.Point(103.2132736942483, 30.275797894943555),ee.Geometry.Point(103.30767286354975, 30.305076352460482),ee.Geometry.Point(104.09084707707171, 30.298797938177234),ee.Geometry.Point(104.08533116906862, 30.399939268271485),ee.Geometry.Point(103.97237768028378, 30.378990411522405),ee.Geometry.Point(103.85999049733272, 30.38616064601634),ee.Geometry.Point(103.64255135266355, 30.380417323859497),ee.Geometry.Point(103.4523305608246, 30.397089902698347),ee.Geometry.Point(103.40188029150592, 30.400621510732663),ee.Geometry.Point(103.28198398238085, 30.42979740892536),ee.Geometry.Point(103.40658464311728, 30.538496915305213),ee.Geometry.Point(103.55018553344246, 30.530787129583803),ee.Geometry.Point(103.75571887829895, 30.51256724267942),ee.Geometry.Point(103.71035198418119, 30.572524315444053),ee.Geometry.Point(104.04451255802537, 30.507523999387978),ee.Geometry.Point(104.07887884984427, 30.541555858627877),ee.Geometry.Point(104.12058036289609, 30.642567874791585),ee.Geometry.Point(103.96430224827598, 30.608977466555977),ee.Geometry.Point(103.7894386004651, 30.64951961333391),ee.Geometry.Point(103.65108607207983, 30.6601902421548),ee.Geometry.Point(103.5123946520145, 30.5943872879091),ee.Geometry.Point(103.40696366670683, 30.591109848634826),ee.Geometry.Point(103.17445721957806, 30.609780104745607),ee.Geometry.Point(103.54244007491263, 30.747655509318147),ee.Geometry.Point(103.66653685723003, 30.88589210107602),ee.Geometry.Point(103.82516045900427, 30.870722239292206),ee.Geometry.Point(103.97441103036414, 30.883854291894597),ee.Geometry.Point(104.30360862473584, 30.87775424165348),ee.Geometry.Point(104.1084560671318, 30.961045790230806),ee.Geometry.Point(104.01300009088068, 31.053331345632905),ee.Geometry.Point(103.83872504440885, 30.999987119730534),ee.Geometry.Point(103.6491463631216, 31.016686453175698),ee.Geometry.Point(103.55922413127782, 31.032137238325884),ee.Geometry.Point(103.82561082821066, 31.157201645571888),ee.Geometry.Point(103.98481411312183, 31.124315775302716),ee.Geometry.Point(103.79927983649007, 31.24365469679609),ee.Geometry.Point(104.39462333633772, 30.77482481203679),ee.Geometry.Point(104.30206131667029, 30.75681004378087),ee.Geometry.Point(104.07915085500854, 30.710257028782898),ee.Geometry.Point(103.92939640522962, 30.74946144524479),ee.Geometry.Point(103.85179912869754, 30.790587734260715),ee.Geometry.Point(103.71332174548279, 30.74727202662755),ee.Geometry.Point(103.11205655943614, 30.3037921313571),ee.Geometry.Point(103.22006044605166, 30.277987313560796),ee.Geometry.Point(103.33819095115955, 30.313584317270458),ee.Geometry.Point(103.68955919596897, 30.31001703642771),ee.Geometry.Point(103.83330723662891, 30.380065054876287),ee.Geometry.Point(104.03116200947136, 30.37966819488253),ee.Geometry.Point(104.01177829719202, 30.287480739703582),ee.Geometry.Point(104.06537223275339, 30.28945166236921),ee.Geometry.Point(104.09395507050597, 30.389402412482195),ee.Geometry.Point(103.69796014235366, 30.386486160393247),ee.Geometry.Point(103.47238759736301, 30.391868295364755),ee.Geometry.Point(103.35112680331558, 30.38069378812482),ee.Geometry.Point(103.22146506288348, 30.38129576676703),ee.Geometry.Point(103.35553685425745, 30.469978368517967),ee.Geometry.Point(103.54630165642493, 30.491123425713408),ee.Geometry.Point(103.66633173858156, 30.511679881569794),ee.Geometry.Point(103.79777711943505, 30.49819110088312),ee.Geometry.Point(103.9886489400278, 30.495930336649014),ee.Geometry.Point(104.12094155008144, 30.50815719173757),ee.Geometry.Point(104.09889129537216, 30.62081637985288),ee.Geometry.Point(104.01699544542456, 30.625400335735815),ee.Geometry.Point(103.81255903942723, 30.648114996502073),ee.Geometry.Point(103.70241478430606, 30.637538008803297),ee.Geometry.Point(103.54247128862, 30.63653025196522),ee.Geometry.Point(103.36022336946462, 30.626787116163445),ee.Geometry.Point(103.22145614468138, 30.62574814561799),ee.Geometry.Point(103.40954994531783, 30.76736473597438),ee.Geometry.Point(103.52267287994272, 30.746759230006408),ee.Geometry.Point(103.64416108814385, 30.75235986092953),ee.Geometry.Point(103.80493843572688, 30.744984507787127),ee.Geometry.Point(103.94081170392643, 30.753028726087546),ee.Geometry.Point(104.155780506612, 30.74467237071339),ee.Geometry.Point(104.21796267080232, 30.74965318659008),ee.Geometry.Point(104.37760294761672, 30.76694558047535),ee.Geometry.Point(104.28252153585409, 30.86377941895198),ee.Geometry.Point(104.14203755716528, 30.848627393572375),ee.Geometry.Point(104.01870328112804, 30.854830003137725),ee.Geometry.Point(103.80665518963245, 30.87030308379318),ee.Geometry.Point(103.67705141751405, 30.8538935919165),ee.Geometry.Point(103.59460263903584, 30.907558873094715),ee.Geometry.Point(103.5942994201642, 30.979644700724716),ee.Geometry.Point(103.63923824058082, 30.97573406910085),ee.Geometry.Point(103.84399570185401, 31.04209441097823),ee.Geometry.Point(103.94716146382653, 31.019861333125753),ee.Geometry.Point(104.12668041313721, 30.994658493971677),ee.Geometry.Point(103.9682663891125, 31.105016785943405),ee.Geometry.Point(103.84483847195311, 31.143128722647205),ee.Geometry.Point(103.66475321680866, 31.119459814255517),ee.Geometry.Point(103.91127907764945, 31.20617149334083)]

In [11]:
result_df_train = pd.DataFrame(columns=['RMSE', 'MAE', 'R2'])
result_df_valid = pd.DataFrame(columns=['RMSE', 'MAE', 'R2'])

for point in points:
    rTable = ee.FeatureCollection(fittedHarmonic1.filterBounds(point).map(lambda image: ee.Feature(
        None,
        {
            'date': image.date().format('yyyy-MM-dd'),
            'NDVI': image.reduceRegion(ee.Reducer.mean(), point, 10).get("NDVI"),
            'fitted': image.reduceRegion(ee.Reducer.mean(), point, 10).get("fitted")
        }
    )))

    rTable_dict = rTable.getInfo()

    dates = [feat['properties']['date'] for feat in rTable_dict['features']]
    ndvi_values = [feat['properties']['NDVI'] for feat in rTable_dict['features']]
    fitted_values = [feat['properties']['fitted'] for feat in rTable_dict['features']]

    df = pd.DataFrame({'Date': dates, 'NDVI': ndvi_values, 'Fitted': fitted_values})

    df = df.dropna()

    if not df.empty:
        df.isetitem(df.columns.get_loc('Date'), pd.to_datetime(df['Date']))

        train_df = df[df['Date'] < '2023-05-01']
        valid_df = df[df['Date'] >= '2023-05-01']

        y_train = train_df['NDVI']
        y_valid = valid_df['NDVI']

        y_train_pred = train_df['Fitted']
        y_valid_pred = valid_df['Fitted']

        rmse_train = mean_squared_error(y_train, y_train_pred, squared=False)
        mae_train = mean_absolute_error(y_train, y_train_pred)
        r2_train = r2_score(y_train, y_train_pred)

        rmse_valid = mean_squared_error(y_valid, y_valid_pred, squared=False)
        mae_valid = mean_absolute_error(y_valid, y_valid_pred)
        r2_valid = r2_score(y_valid, y_valid_pred)

        result_df_train = pd.concat([result_df_train, pd.DataFrame([{'RMSE': rmse_train, 'MAE': mae_train, 'R2': r2_train}])], ignore_index=True)
        result_df_valid = pd.concat([result_df_valid, pd.DataFrame([{'RMSE': rmse_valid, 'MAE': mae_valid, 'R2': r2_valid}])], ignore_index=True)

rmse_train = np.mean(result_df_train['RMSE'])
mae_train = np.mean(result_df_train['MAE'])
r2_train = np.mean(result_df_train['R2'])
print(rmse_train, mae_train, r2_train)

rmse_valid = np.mean(result_df_valid['RMSE'])
mae_valid = np.mean(result_df_valid['MAE'])
r2_valid = np.mean(result_df_valid['R2'])
print(rmse_valid, mae_valid, r2_valid)

0.05162703147480047 0.03761068014404352 0.8392909154974886
899.2946138159385 592.7032098982784 -4425507883.152811


In [12]:
result_df_train = result_df_train.dropna()
result_df_valid = result_df_valid.dropna()

In [13]:
rmse_train = np.median(result_df_train['RMSE'])
mae_train = np.median(result_df_train['MAE'])
r2_train = np.median(result_df_train['R2'])
print(rmse_train, mae_train, r2_train)

rmse_valid = np.median(result_df_valid['RMSE'])
mae_valid = np.median(result_df_valid['MAE'])
r2_valid = np.median(result_df_valid['R2'])
print(rmse_valid, mae_valid, r2_valid)

0.048782221877036264 0.03627071467386027 0.8879199483895149
0.272744128707798 0.1985455800773751 -3.7050483901979594
